In [1]:
import numpy as np
import cv2

In [89]:
def truncate_descriptor(descriptors, degree):
    """this function truncates an unshifted fourier descriptor array
    and returns one also unshifted"""
    if degree == -1:
        return descriptors
    descriptors = np.fft.fftshift(descriptors)
    center_index = len(descriptors) // 2
    descriptors = descriptors[
        int(np.ceil(center_index-degree/2)):int(np.ceil(center_index+degree/2))]
    descriptors = np.fft.ifftshift(descriptors)
    return descriptors

def reconstruct(descriptors, degree):
    """ reconstruct(descriptors, degree) attempts to reconstruct the image
    using the first [degree] descriptors of descriptors"""
    # truncate the long list of descriptors to certain length
    descriptor_in_use = truncate_descriptor(descriptors, degree)
    contour_reconstruct = np.fft.ifft(descriptor_in_use)
    contour_reconstruct = np.array(
        [contour_reconstruct.real, contour_reconstruct.imag])
    contour_reconstruct = np.transpose(contour_reconstruct)
    contour_reconstruct = np.expand_dims(contour_reconstruct, axis=1)
    # make positive
    if contour_reconstruct.min() < 0:
        contour_reconstruct -= contour_reconstruct.min()
    # normalization
    contour_reconstruct *= 300 / contour_reconstruct.max()
    # type cast to int32
    contour_reconstruct = contour_reconstruct.astype(np.int32, copy=False)
    black = np.zeros((400, 400), np.uint8)
    # draw and visualize
    cv2.drawContours(black, contour_reconstruct, -1, 255, thickness=2)
    cv2.imshow("black", black)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return descriptor_in_use

In [69]:
def flip(cnts, hierarchy, mask):
    cnts_new = np.empty(len(mask), dtype=object)
    for i in range(len(mask)):
        need_flip = False
        index = hierarchy[0, mask[i], 3]
        while index != -1:
            index = hierarchy[0, index, 3]
            need_flip = not need_flip
        if need_flip:
            cnts_new[i] = np.flip(cnts[mask[i]], 0)
        else:
            cnts_new[i] = cnts[mask[i]]
    return cnts_new

def contour(img, mode='c'):
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if mode == 'c':
        ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    else:
        thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 81, -5)
    _, cnts, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    mask = range(len(cnts))
    if mode == 'q':
        threshold = [80, 400]
        mask = np.where(np.array([threshold[0]<len(n)<threshold[1] for n in cnts]) > 0)[0]
#     outer: counter-clockwise, inner: clockwise, need to flip to align to counter-clockwise
    print('Contour size:', np.asarray(cnts).shape)
    cnts = flip(cnts, hierarchy, mask)
    return cnts

def fourier_descriptor(cnts):
    fd = []
    for i in cnts:
        contour_array = i[:, 0, :]
        contour_complex = np.empty(contour_array.shape[:-1], dtype=complex)
        contour_complex.real = contour_array[:, 0]
        contour_complex.imag = contour_array[:, 1]
        fourier_result = np.fft.fft(contour_complex)
        fd.append(fourier_result)
    return fd

def translation(fd):
    t = np.array([fd[0].real, fd[0].imag])
    fd[0] = 0
    return fd, t

def scaling(fd):
    s = np.linalg.norm(fd)
    fd /= s
    return fd, s

def invarient_fd(fd, shorter):
    fd_new = truncate_descriptor(fd, shorter)
    fd_new, t = translation(fd_new)
    fd_new, s = scaling(fd_new)
    c = np.fft.ifft(fd_new)
    return fd_new, c, t, s

def geometry(img, cnts):
    v = []
    center = np.array([img.shape[0]/2, img.shape[1]/2])
    for cnt in cnts:
        local = np.zeros(2)
        for pair in cnt:
            local += pair[0]
        local /= len(cnt)
        v.append(local-center)
    return v

In [70]:
def correlation(IMG_Q, img_C):
    img_Q = cv2.imread(IMG_Q['IMG'])
    img_C = cv2.imread(IMG_C['IMG'])
    cnts_Q = contour(img_Q, IMG_Q['mode'])
    cnts_C = contour(img_C, IMG_C['mode'])
#     seems not invarient to index-shift 
#     cnts_Q[0] = np.roll(cnts_Q[0], 50, 0)
    v_C = geometry(img_C, cnts_C)
    fd_Q = fourier_descriptor(cnts_Q)
    fd_C = fourier_descriptor(cnts_C)
    shape = np.array([len(fd_Q), len(fd_C)])
    correlate_QC = np.zeros(shape)
    s_QC = np.zeros(shape)
    t_QC = np.zeros(np.append(shape, 2))
    for j in range(shape[0]):
        for k in range(shape[1]):
#             pick the first several most important points
            shorter = min(len(fd_Q[j]), len(fd_C[k]))
            fd_Q_j, c_Q, t_Q, s_Q = invarient_fd(fd_Q[j], shorter)
            fd_C_k, c_C, t_C, s_C = invarient_fd(fd_C[k], shorter)
            s_QC[j, k] = s_Q / s_C
            t_QC[j, k] = t_Q - t_C
#             if we want to neglect index-shift, we need to roll the original contour and pick the same length fd
            correlate = np.abs(np.correlate(c_Q, c_C))*shorter
#             print(correlate)
            correlate = 2 - 2 * correlate
            correlate_QC[j, k] = correlate
#     need element-wise threshold for each prototype contour
    M = np.where(correlate_QC < 0.2)
    fit = []
    s_cut = 0.8
    d_cut = 0.7
    for i in range(len(M[0])):
        j = M[0][i]
        k = M[1][i]
        v_jk = s_QC[j, k] * v_C[k] + t_QC[j, k]
        pair = [j]
        for i_ in range(len(M[0])):
            j_ = M[0][i_]
            k_ = M[1][i_]
            if k_ == k or j_ == j:
                continue
#             some problem in scaling and translation
            s_similar = s_QC[j_, k_] / s_QC[j, k]
            v_j_k_ = s_QC[j_, k_] * v_C[k_] + t_QC[j_, k_]
            d_similar = v_j_k_ / v_jk
            if s_cut<s_similar<1/s_cut and d_cut<d_similar.min() and d_similar.max()<1/d_cut:
                pair.append(j_)
#                 print('-'*80)
#                 print(j, k)
#                 print(j_, k_)
#                 print('s_similar:', s_similar)
#                 print('d_similar:', d_similar)
        if len(pair) >= shape[1]:
            fit.append(pair)
#             print('pair:', pair)
    if IMG_Q['draw']:
        img = cv2.drawContours(img_Q ,cnts_Q, -1, (0,255,255), 2)
        for i in M[0]:
            img = cv2.drawContours(img ,cnts_Q, i, (0,0,255), 2)
        for pair in fit:
            for i in pair:
                img = cv2.drawContours(img ,cnts_Q, i, (0,255,0), 2)
        img = cv2.resize(img, (int(img.shape[1]*11/16), int(img.shape[0]*11/16)))
        cv2.imshow('img', img)
        cv2.waitKey()
        cv2.destroyAllWindows()
    if IMG_C['draw']:
        img = cv2.drawContours(img_C ,cnts_C, -1, (0,255,255), 2)
        img = cv2.resize(img, (int(img.shape[1]*11/16), int(img.shape[0]*11/16)))
        cv2.imshow('img', img)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return correlate_QC

In [71]:
# Query image + Contour image
IMG_Q = {'IMG':'fd_image/traffic5.jpg', 'mode':'q', 'draw':True}
IMG_C = {'IMG':'fd_image/sign1.jpg', 'mode':'c', 'draw':False}
c = correlation(IMG_Q, IMG_C)

Contour size: (5154,)
Contour size: (2,)


# Hyperparameters

In [ ]:
# color: 81, -5
# threshold: 100, 500
traffic3: 9, 10 (0.2, 0.6, 0.8)
traffic4: 5, 6 (0.2, 0.7, 0.7)
-------------------
# threshold: 80, 400
traffic5: 30, 31 (0.2, 0.8, 0.7)
-------------------
# threshold: 300, 1200
traffic6: 12, 13 (0.3, -, -)
--------------------------------------
# color: 41, 5
# threshold: 100, 500
traffic7: (0.2, 0.8, 0.7, -2)

# Remain to be done

1. how to solve the situation when c1 and c2 have different length
2. how to align the center using scaling and translation
3. how to extract the contour precisely and robustly
4. how to define the lower and upper bound of contour
5. how to do correlation with different step length
6. how to tune the similarity of different j and k
7. how to make it index-shift invariant
8. how to match the affine image
9. more efficient cascaded matching scheme 